In [7]:
import numpy as np
import pandas as pd
import math

In [8]:
def load_weights_and_biases():
    gates = ['i', 'f', 'o', 'c']
    ih_weights, hh_weights, ih_biases, hh_biases = {}, {}, {}, {}

    for gate in gates:
        ih_weights[gate] = pd.read_csv(f"LSTM_ih_weights_{gate}.csv").values
        hh_weights[gate] = pd.read_csv(f"LSTM_hh_weights_{gate}.csv").values
        ih_biases[gate] = pd.read_csv(f"LSTM_ih_biases_{gate}.csv").values.flatten()
        hh_biases[gate] = pd.read_csv(f"LSTM_hh_biases_{gate}.csv").values.flatten()

    linear_weights = pd.read_csv("Linear_weights.csv").values
    linear_bias = pd.read_csv("Linear_bias.csv").values.flatten()

    return ih_weights, hh_weights, ih_biases, hh_biases, linear_weights, linear_bias

ih_weights, hh_weights, ih_biases, hh_biases, linear_weights, linear_bias = load_weights_and_biases()


In [9]:
class SimpleLSTM:
    def __init__(self, ih_weights, hh_weights, ih_biases, hh_biases, linear_weights, linear_bias):
        self.ih_weights = {key: val.T for key, val in ih_weights.items()}  # Transpose weights
        self.hh_weights = {key: val.T for key, val in hh_weights.items()}
        self.ih_biases = ih_biases
        self.hh_biases = hh_biases
        self.linear_weights = linear_weights
        self.linear_bias = linear_bias

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def tanh(self, x):
        return np.tanh(x)

    def forward(self, x):
        h_prev = np.zeros((x.shape[0], self.hh_weights['i'].shape[1]))
        c_prev = np.zeros((x.shape[0], self.hh_weights['i'].shape[1]))

        for time_step in range(x.shape[1]):
            x_t = x[:, time_step, :]

            i = self.sigmoid(np.dot(x_t, self.ih_weights['i']) + np.dot(h_prev, self.hh_weights['i']) + self.ih_biases['i'] + self.hh_biases['i'])
            f = self.sigmoid(np.dot(x_t, self.ih_weights['f']) + np.dot(h_prev, self.hh_weights['f']) + self.ih_biases['f'] + self.hh_biases['f'])
            o = self.sigmoid(np.dot(x_t, self.ih_weights['o']) + np.dot(h_prev, self.hh_weights['o']) + self.ih_biases['o'] + self.hh_biases['o'])
            g = self.tanh(np.dot(x_t, self.ih_weights['c']) + np.dot(h_prev, self.hh_weights['c']) + self.ih_biases['c'] + self.hh_biases['c'])

            c_curr = f * c_prev + i * g
            h_curr = o * self.tanh(c_curr)

            h_prev, c_prev = h_curr, c_curr

        output = np.dot(h_curr, self.linear_weights.T) + self.linear_bias  # Ensure linear weights are correctly aligned
        return output


In [10]:
data_day_minus1 = pd.read_csv('/Users/oscar/Library/CloudStorage/OneDrive-Personal/Documents/PERSONAL/Finance/Competition/IMC Prosperity/IMC_Prosperity_Huskies/Round 2/round-2-island-data-bottle/prices_round_2_day_-1.csv', sep=';')
data_day_0 = pd.read_csv('/Users/oscar/Library/CloudStorage/OneDrive-Personal/Documents/PERSONAL/Finance/Competition/IMC Prosperity/IMC_Prosperity_Huskies/Round 2/round-2-island-data-bottle/prices_round_2_day_0.csv', sep=';')
data_day_plus1 = pd.read_csv('/Users/oscar/Library/CloudStorage/OneDrive-Personal/Documents/PERSONAL/Finance/Competition/IMC Prosperity/IMC_Prosperity_Huskies/Round 2/round-2-island-data-bottle/prices_round_2_day_1.csv', sep=';')

In [11]:
train_data_0 = data_day_0[['ORCHIDS', 'TRANSPORT_FEES', 'EXPORT_TARIFF', 'IMPORT_TARIFF', 'SUNLIGHT', 'HUMIDITY']]
train_data_0.index = data_day_0.timestamp

train_data_m1 = data_day_minus1[['ORCHIDS', 'TRANSPORT_FEES', 'EXPORT_TARIFF', 'IMPORT_TARIFF', 'SUNLIGHT', 'HUMIDITY']]
train_data_m1.index = data_day_minus1.timestamp

train_data_p1 = data_day_plus1[['ORCHIDS', 'TRANSPORT_FEES', 'EXPORT_TARIFF', 'IMPORT_TARIFF', 'SUNLIGHT', 'HUMIDITY']]
train_data_p1.index = data_day_plus1.timestamp


In [26]:
Mu_day0 = np.array([ 1.04125117e+03,  1.38491151e+00,  9.96115388e+00, -2.57076292e+00,
        2.27953335e+03,  7.40365235e+01])

Sigma_day0 = np.array([4.51427120e+01, 1.93678379e-01, 1.58433451e+00, 3.49320924e-01,
       8.13568530e+02, 1.01992251e+01])

N_train_data_0 = (train_data_0 - Mu_day0) / Sigma_day0
N_train_data_m1 = (train_data_m1 - Mu_day0) / Sigma_day0
N_train_data_p1 = (train_data_p1 - Mu_day0) / Sigma_day0

In [27]:
N_train_data_p1

,ORCHIDS,TRANSPORT_FEES,EXPORT_TARIFF,IMPORT_TARIFF,SUNLIGHT,HUMIDITY
timestamp,,,,,,
0,1.262636,-2.503695,0.340109,-6.95417,-0.220674,-0.395768
100,1.262636,-2.503695,-0.291071,-6.95417,-0.220973,-0.393541
200,1.229408,-2.503695,-0.291071,-6.95417,-0.221272,-0.391317
300,1.234946,-2.503695,-0.291071,-6.95417,-0.221573,-0.389098
400,1.240484,-2.503695,-0.291071,-6.95417,-0.221875,-0.386881
...,...,...,...,...,...,...
999600,-0.110786,-2.503695,-0.291071,-6.95417,0.025087,-0.402093
999700,-0.110786,-2.503695,-0.291071,-6.95417,0.025102,-0.400520
999800,-0.110786,-2.503695,-0.291071,-6.95417,0.025119,-0.398942


In [14]:
sequence_length = 15
index = np.random.randint(0,5000)
test_input = np.array([N_train_data_p1.iloc[index:index+sequence_length].values])

In [15]:
# Assuming input X is loaded or created with appropriate shape [batch_size, sequence_length, input_size]
model = SimpleLSTM(ih_weights, hh_weights, ih_biases, hh_biases, linear_weights, linear_bias)


In [16]:
# Assuming the SimpleLSTM class and the weight variables are defined and initialized as shown previously
output = model.forward(test_input)


In [17]:
output

array([[ 0.57005998, -0.84870977, -0.13065206, -0.77768019, -0.55007663,
         0.19743686]])

In [18]:
print("Input-to-hidden weights shape (i):", model.ih_weights['i'].shape)
print("Hidden-to-hidden weights shape (i):", model.hh_weights['i'].shape)
print("Input shape:", test_input.shape)


Input-to-hidden weights shape (i): (6, 10)
Hidden-to-hidden weights shape (i): (10, 10)
Input shape: (1, 15, 6)
